In [235]:
import pytesseract as pt
from PIL import Image
import numpy as np
import cv2
import re
import os
import sys

# CONFIGURAR CAMINHO PARA O TESSERACT

In [70]:
pt.pytesseract.tesseract_cmd = "C:/Users/c18361b/AppData/Local/Programs/Tesseract-OCR/tesseract.exe"

# VARIÁVEIS GLOBAIS

In [288]:
FILEPATH = f'./carPlates/'
#FILEPATH = f'C:/Users/c18361b/Downloads/'

IMAGE = 'plate_3.bmp'

IMAGES = ['plate_1.bmp', 'plate_2.bmp', 'plate_3.bmp', 'plate_4.bmp', 'plate_5.bmp']

# DETECTAR CARACTERES DA PLACA COM TESSERACT

In [281]:
def chrDetector(image):
    
    # usa o pytesseract para detectar os caracteres e obter informações sobre eles
    text = pt.pytesseract.image_to_string(image)
    chr_data = pt.pytesseract.image_to_boxes(image)

    # define o padrão a ser procurado (3 números seguidos por 4 letras)
    sequence = (r"\b(?=.*[A-Z])(?=.*\d)[A-Z\d]{3}.*\d{4}\b|"
                r"\b(?=.*[A-Z])(?=.*\d)[A-Z]{3}.*[A-Z\d]{4}\b|"
                r"\b(?=.*[A-Z])(?=.*\d)[A-Z]{3}.*\d{4}\b|"
                r"\b(?=.*[A-Z])(?=.*\d)[A-Z\d]{3}.*[A-Z\d]{3}\b")

    # encontra todos os padrões no texto e junta em uma lista
    results = re.findall(sequence, text)

    # separa cada caracter de results e coloca em uma nova lista
    plate_chr = []
    for chr in results:
        plate_chr.extend(list(chr))
    
    # relaciona os itens de plate_chr com os detectados pelo tesseract para extrair as informações
    # apenas dos caracteres desejados
    chr_coord = []
    for info in chr_data.splitlines():
        char, x, y, w, h = info.split()[0], int(info.split()[1]), int(info.split()[2]), int(info.split()[3]), int(info.split()[4])
        if char.isalnum() and char in plate_chr:  # Verificar se o caractere é alfanumérico
            chr_coord.append({'char': char, 'x': x, 'y': y, 'width': w, 'height': h})

    # mantém na lista apenas os caracteres necessários e exclui os extras indesejados
    del chr_coord[:-7]

    # exibe as coordenadas dos caracteres correspondentes aos padrões encontrados
    """for data in chr_coord:
        print(f"Caractere: {data['char']}, Coordenadas: x={data['x']}, y={data['y']}, largura={data['width']}, altura={data['height']}")
"""
    return chr_coord

# CORTANDO OS CARACTERES DA IMAGEM

In [280]:
def cropCharacters(image, chr_coord, which_plate):

    height, width = image.shape
    
    for index, item in enumerate(chr_coord):

        offset = 10

        x_top_left = int(item['x']) - offset
        y_top_left = int(height - int(item['height'])) - offset
        x_bottom_right = int(item['width']) + offset
        y_bottom_right = int(height - int(item['y'])) + offset

        while (x_top_left < 0 or y_top_left < 0 or x_bottom_right > width or y_bottom_right > height) and offset > 0:
            offset -= 1
            if x_top_left < 0: x_top_left = int(item['x']) - offset
            if y_top_left < 0: y_top_left = int(height - int(item['height'])) - offset
            if x_bottom_right > width: x_bottom_right = int(item['width']) + offset
            if y_bottom_right > height: y_bottom_right = int(height - int(item['y'])) + offset

        cropped_img = image[y_top_left:y_bottom_right, x_top_left:x_bottom_right]

        directory = './carPlates/cropped_plate_%d'% which_plate
        if not os.path.exists(directory): os.makedirs(directory)

        cropped_img = Image.fromarray(cropped_img)
        cropped_img.save(os.path.join(directory, 'chr_%d.png'% index))

# MAIN

In [291]:
def main():

    for index, img in enumerate(IMAGES):
        
        img_path = FILEPATH + img
        #img_path = f'./carPlates/' + IMAGE
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        chr_coord = chrDetector(image)
        cropCharacters(image, chr_coord, which_plate=index+1)

        #print("Done %d"% index)
           
if __name__ == '__main__':
    main()

Done 0
Done 1
Done 2
Done 3
Done 4
